In [1]:
import findspark

In [2]:
findspark.init('')

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Retail").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/22 09:16:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/09/22 09:16:16 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
sc = spark.sparkContext

In [5]:
# Reading the retail dataset
cust = spark.read.option("inferSchema",True).option("header",True).option("sep", '\t').csv("/home/labuser/Downloads/Retail_Dataset/customer.csv")
cust.printSchema()

root
 |-- C_CUSTKEY: integer (nullable = true)
 |-- C_NAME: string (nullable = true)
 |-- C_ADDRESS: string (nullable = true)
 |-- C_NATIONKEY: integer (nullable = true)
 |-- C_PHONE: string (nullable = true)
 |-- C_ACCTBAL: double (nullable = true)
 |-- C_MKTSEGMENT: string (nullable = true)
 |-- C_COMMENT: string (nullable = true)



In [6]:
line = spark.read.option("inferSchema",True).option("header",True).option("sep", '\t').csv("/home/labuser/Downloads/Retail_Dataset/lineitem.csv")
line.printSchema()

root
 |-- L_ORDERKEY: integer (nullable = true)
 |-- L_PARTKEY: integer (nullable = true)
 |-- L_SUPPKEY: integer (nullable = true)
 |-- L_LINENUMBER: integer (nullable = true)
 |-- L_QUANTITY: double (nullable = true)
 |-- L_EXTENDEDPRICE: double (nullable = true)
 |-- L_DISCOUNT: double (nullable = true)
 |-- L_TAX: double (nullable = true)
 |-- L_RETURNFLAG: string (nullable = true)
 |-- L_LINESTATUS: string (nullable = true)
 |-- L_SHIPDATE: date (nullable = true)
 |-- L_COMMITDATE: date (nullable = true)
 |-- L_RECEIPTDATE: date (nullable = true)
 |-- L_SHIPINSTRUCT: string (nullable = true)
 |-- L_SHIPMODE: string (nullable = true)
 |-- L_COMMENT: string (nullable = true)



In [7]:
nation = spark.read.option("inferSchema",True).option("header",True).option("sep", '\t').csv("/home/labuser/Downloads/Retail_Dataset/nation.csv")
nation.printSchema()

root
 |-- N_NATIONKEY: integer (nullable = true)
 |-- N_NAME: string (nullable = true)
 |-- N_REGIONKEY: integer (nullable = true)
 |-- N_COMMENT: string (nullable = true)



In [8]:
orders = spark.read.option("inferSchema",True).option("header",True).option("sep", '\t').csv("/home/labuser/Downloads/Retail_Dataset/orders.csv")
orders.printSchema()

root
 |-- O_ORDERKEY: integer (nullable = true)
 |-- O_CUSTKEY: integer (nullable = true)
 |-- O_ORDERSTATUS: string (nullable = true)
 |-- O_TOTALPRICE: double (nullable = true)
 |-- O_ORDERDATE: date (nullable = true)
 |-- O_ORDERPRIORITY: string (nullable = true)
 |-- O_CLERK: string (nullable = true)
 |-- O_SHIPPRIORITY: integer (nullable = true)
 |-- O_COMMENT: string (nullable = true)



In [9]:
part = spark.read.option("inferSchema",True).option("header",True).option("sep", '\t').csv("/home/labuser/Downloads/Retail_Dataset/part.csv")
part.printSchema()

root
 |-- P_PARTKEY: integer (nullable = true)
 |-- P_NAME: string (nullable = true)
 |-- P_MFGR: string (nullable = true)
 |-- P_BRAND: string (nullable = true)
 |-- P_TYPE: string (nullable = true)
 |-- P_SIZE: integer (nullable = true)
 |-- P_CONTAINER: string (nullable = true)
 |-- P_RETAILPRICE: double (nullable = true)
 |-- P_COMMENT: string (nullable = true)



In [10]:
partsupp = spark.read.option("inferSchema",True).option("header",True).option("sep", '\t').csv("/home/labuser/Downloads/Retail_Dataset/partsupp.csv")
partsupp.printSchema()

root
 |-- PS_PARTKEY: integer (nullable = true)
 |-- PS_SUPPKEY: integer (nullable = true)
 |-- PS_AVAILQTY: integer (nullable = true)
 |-- PS_SUPPLYCOST: double (nullable = true)
 |-- PS_COMMENT: string (nullable = true)



In [11]:
region = spark.read.option("inferSchema",True).option("header",True).option("sep", '\t').csv("/home/labuser/Downloads/Retail_Dataset/region.csv")
region.printSchema()

root
 |-- R_REGIONKEY: integer (nullable = true)
 |-- R_NAME: string (nullable = true)
 |-- R_COMMENT: string (nullable = true)



In [12]:
supplier = spark.read.option("inferSchema",True).option("header",True).option("sep", '\t').csv("/home/labuser/Downloads/Retail_Dataset/supplier.csv")
supplier.printSchema()

root
 |-- S_SUPPKEY: integer (nullable = true)
 |-- S_NAME: string (nullable = true)
 |-- S_ADDRESS: string (nullable = true)
 |-- S_NATIONKEY: integer (nullable = true)
 |-- S_PHONE: string (nullable = true)
 |-- S_ACCTBAL: double (nullable = true)
 |-- S_COMMENT: string (nullable = true)



In [19]:
joindf = cust.join(orders,cust.C_CUSTKEY == orders.O_CUSTKEY, how = "inner").\
         join(line, orders.O_ORDERKEY == line.L_ORDERKEY, how = "inner").\
         select(cust["*"],orders["O_TOTALPRICE"].alias("orderprice"),orders["O_ORDERSTATUS"],line["L_LINENUMBER"],line["L_LINESTATUS"])
joindf.show()

+---------+------------------+--------------------+-----------+---------------+---------+------------+--------------------+----------+-------------+------------+------------+
|C_CUSTKEY|            C_NAME|           C_ADDRESS|C_NATIONKEY|        C_PHONE|C_ACCTBAL|C_MKTSEGMENT|           C_COMMENT|orderprice|O_ORDERSTATUS|L_LINENUMBER|L_LINESTATUS|
+---------+------------------+--------------------+-----------+---------------+---------+------------+--------------------+----------+-------------+------------+------------+
|      370|Customer#000000370|          oyAPndV IN|         12|22-524-280-8721|  8982.79|   FURNITURE|ges. final packag...| 172799.49|            O|           6|           O|
|      370|Customer#000000370|          oyAPndV IN|         12|22-524-280-8721|  8982.79|   FURNITURE|ges. final packag...| 172799.49|            O|           5|           O|
|      370|Customer#000000370|          oyAPndV IN|         12|22-524-280-8721|  8982.79|   FURNITURE|ges. final packag...| 1